In [1]:
import pandas as pd
import numpy as np
import json
import re
import xml.etree.ElementTree as ET
from statsmodels.stats.contingency_tables import mcnemar
from statsmodels.stats.contingency_tables import cochrans_q
from scipy.stats import combine_pvalues
from scipy.stats import norm

In [2]:
def read_xml_answers(file_path):
    tree = ET.parse(file_path)
    root = tree.getroot()

    answers = []
    for obj in root.findall("root"):
        answer = obj.find("answer")
        if answer is not None:
            number = str(answer.text).replace(',', '')
            number = re.findall(r'\d+\.\d+|\d+', number)
            answers.append(number[-1])
    return answers

def read_unstructured_parsed(file_path):
    with open(file_path, "r") as f:
        data = json.load(f)

    answers = []
    if isinstance(data, list):
        for obj in data:
            if isinstance(obj, dict) and "parsed" in obj:
                number = str(obj["parsed"]).replace(',', '')
                number = re.findall(r'\d+\.\d+|\d+', number)
                answers.append(number[0])
    return answers

In [3]:
# Structured answers
xml_standard_answers = read_xml_answers("gsm8k_results/test_top200_xml_0.xml")
xml_variant_0_answers = read_xml_answers("gsm8k_results/test_top50_variant_0_xml_0.xml")
xml_variant_1_answers = read_xml_answers("gsm8k_results/test_top50_variant_0_xml_1.xml")
xml_variant_2_answers = read_xml_answers("gsm8k_results/test_top50_variant_0_xml_2.xml")
# Unstructured answers
unstructured_standard_answers = read_unstructured_parsed("gsm8k_results/test_top200_unstructured_parsed_0.json")
unstructured_variant_0_answers = read_unstructured_parsed("gsm8k_results/test_top50_variant_0_unstructured_parsed_0.json")
unstructured_variant_1_answers = read_unstructured_parsed("gsm8k_results/test_top50_variant_1_unstructured_parsed_0.json")
unstructured_variant_2_answers = read_unstructured_parsed("gsm8k_results/test_top50_variant_2_unstructured_parsed_0.json")

In [4]:
with open("data/gsm8k_test.json", "r") as f:
    label = json.load(f)
unstructured_standard_results = []
unstructured_variant_0_results = []
unstructured_variant_1_results = []
unstructured_variant_2_results = []

xml_standard_results = []
xml_variant_0_results = []
xml_variant_1_results = []
xml_variant_2_results = []

for i in range(len(unstructured_variant_0_answers)):
    label_tmp = float(label[i]['answer'].split("\n#### ")[-1].replace(',', ''))
    # Unstructured answers
    if label_tmp == float(unstructured_standard_answers[i]):
        unstructured_standard_results.append(1)
    else:
        unstructured_standard_results.append(0)
        
    if label_tmp == float(unstructured_variant_0_answers[i]):
        unstructured_variant_0_results.append(1)
    else:
        unstructured_variant_0_results.append(0)
        
    if label_tmp == float(unstructured_variant_1_answers[i]):
        unstructured_variant_1_results.append(1)
    else:
        unstructured_variant_1_results.append(0)
        
    if label_tmp == float(unstructured_variant_2_answers[i]):
        unstructured_variant_2_results.append(1)
    else:
        unstructured_variant_2_results.append(0)
    
    # Structured answers
    if label_tmp == float(xml_standard_answers[i]):
        xml_standard_results.append(1)
    else:
        xml_standard_results.append(0)   
        
    if label_tmp == float(xml_variant_0_answers[i]):
        xml_variant_0_results.append(1)
    else:
        xml_variant_0_results.append(0) 
        
    if label_tmp == float(xml_variant_1_answers[i]):
        xml_variant_1_results.append(1)
    else:
        xml_variant_1_results.append(0) 
        
    if label_tmp == float(xml_variant_2_answers[i]):
        xml_variant_2_results.append(1)
    else:
        xml_variant_2_results.append(0)

In [5]:
# Test whether output format significantly influences LLMs' generation
print(f"Averaged accuracy of unstructured format with diverse instruction: {format((sum(unstructured_standard_results)+sum(unstructured_variant_0_results)+sum(unstructured_variant_1_results)+sum(unstructured_variant_2_results))/(4*len(unstructured_standard_results)),'.3f')}")
print(f"Averaged accuracy of XML format with diverse instruction: {format((sum(xml_standard_results)+sum(xml_variant_0_results)+sum(xml_variant_1_results)+sum(xml_variant_2_results))/(4*len(xml_standard_results)),'.3f')}")
data_subsets = [
    # Stratum of control on standard instruction
    (np.array(xml_standard_results), np.array(unstructured_standard_results)),
    # Stratum of control on 0 instruction
    (np.array(xml_variant_0_results), np.array(unstructured_variant_0_results)),
    # Stratum of control on 1 instruction
    (np.array(xml_variant_1_results), np.array(unstructured_variant_1_results)),
    # Stratum of control on 2 instruction
    (np.array(xml_variant_2_results), np.array(unstructured_variant_2_results)),
]

p_values = []

for i, (correct_D, correct_E) in enumerate(data_subsets, 1):
    # Build contingency table for this stratum
    table = np.zeros((2, 2), dtype=int)
    for d, e in zip(correct_D, correct_E):
        table[d, e] += 1

    # Run McNemar's test
    result = mcnemar(table, exact=True)
    p_values.append(result.pvalue)

    # print(f"Stratum {i} contingency table:\n{table}")
    # print(f"Stratum {i} McNemar p-value: {result.pvalue:.5f}\n")

# Combine p-values using Stouffer’s method with correlation adjustment
# Convert p-values to Z-scores (two-sided)
z_scores = norm.isf(np.array(p_values) / 2)  # inverse survival function

# Assume weights (e.g., uniform or by number of samples per stratum)
weights = np.ones_like(z_scores)
# Assume correlation between strata (due to shared samples)
rho = 0.3
k = len(p_values)
R = np.full((k, k), rho)
np.fill_diagonal(R, 1)  # correlation matrix

# Stouffer's Z with correlation adjustment
numerator = np.sum(weights * z_scores)
denominator = np.sqrt(np.dot(weights, R @ weights))
z_combined = numerator / denominator
p_combined = 2 * norm.sf(abs(z_combined))

# print(f"Combined Stouffer Z: {z_combined:.3f}")
print(f"Combined p-value across strata (Stouffer’s method): {p_combined:.3f}")

Averaged accuracy of unstructured format with diverse instruction: 0.930
Averaged accuracy of XML format with diverse instruction: 0.955
Combined p-value across strata (Stouffer’s method): 0.625


In [6]:
# Test whether instruction significantly influences LLMs' generation
print(f"Averaged accuracy of unstructured & XML format with standard instruction: {format((sum(unstructured_standard_results)+sum(xml_standard_results))/(2*len(xml_standard_results)),'.3f')}")
print(f"Averaged accuracy of unstructured & XML format with intervened instruction: {format((sum(unstructured_variant_0_results)+sum(unstructured_variant_1_results)+sum(unstructured_variant_2_results)+sum(xml_variant_0_results)+sum(xml_variant_1_results)+sum(xml_variant_2_results))/(6*len(xml_standard_results)),'.3f')}")

stratum_xml = np.array([xml_standard_results, xml_variant_0_results, xml_variant_1_results, xml_variant_2_results]).transpose()
stratum_unstructured = np.array([unstructured_standard_results, unstructured_variant_0_results, unstructured_variant_1_results, unstructured_variant_2_results]).transpose()
data_strata = [stratum_xml, stratum_unstructured]

# Run Cochran's Q test on each stratum
p_values = []
# print("Cochran's Q test per stratum:")
for i, data in enumerate(data_strata):
    result = cochrans_q(data)
    p_values.append(result.pvalue)
    # print(f"  Stratum {i+1}: Q = {result.statistic:.4f}, p = {result.pvalue:.5f}")
    
# Combine p-values using Stouffer’s method with correlation adjustment
# Convert p-values to Z-scores (two-sided)
z_scores = norm.isf(np.array(p_values) / 2)  # inverse survival function

# Assume weights (e.g., uniform or by number of samples per stratum)
weights = np.ones_like(z_scores)
# Assume correlation between strata (due to shared samples)
rho = 0.3
k = len(p_values)
R = np.full((k, k), rho)
np.fill_diagonal(R, 1)  # correlation matrix

# Stouffer's Z with correlation adjustment
numerator = np.sum(weights * z_scores)
denominator = np.sqrt(np.dot(weights, R @ weights))
z_combined = numerator / denominator
p_combined = 2 * norm.sf(abs(z_combined))

# print(f"Combined Stouffer Z: {z_combined:.3f}")
print(f"Combined p-value across strata (Stouffer’s method): {p_combined:.3f}")

Averaged accuracy of unstructured & XML format with standard instruction: 0.960
Averaged accuracy of unstructured & XML format with intervened instruction: 0.937
Combined p-value across strata (Stouffer’s method): 0.118
